## Useful links:

- [Dataset documentation](https://nijianmo.github.io/amazon/index.html)
- [Complete Metadata files](http://deepyeti.ucsd.edu/jianmo/amazon/index.html)
- [Pandas reference sheet](https://ds100.org/sp21/resources/assets/exams/sp20/sp20_checkpoint_reference_sheet.pdf)
- [Data-200 Google Doc](https://docs.google.com/document/d/19HWODy5kpWoUB7BEKEmKLbRnK8MC1fBmRat_WP7vfNc/edit)
- [Grad Project Guidelines](https://ds100.org/sp21/grad_proj/gradproject/)

## Utils

In [ ]:
# Imports.

import os
import json
import gzip
import urllib.request
from urllib.request import urlopen
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn import model_selection

pd.options.mode.chained_assignment = None

In [ ]:
# Utils.

#################################### Loading data. ####################################

def load_data(url, filename):
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url,filename)
        
    # Load the data.
    data = []
    with gzip.open(filename) as f:
        for l in f:
            data.append(json.loads(l.strip()))
    
    df = pd.DataFrame.from_dict(data)
    print('data shape:', df.shape)
    print('first rows of data:')
    display(df.head(3))
    return df

def get_metadata_with_ratings(reviews, metadata):
    ratings = reviews[['asin', 'overall']].groupby('asin').agg('mean').rename(columns={'overall': 'rating'})
    metadata_with_ratings = metadata.merge(ratings, how="left", on="asin")
    
    # Check how many products have ratings.
    print('distribution of ratings:')
    display(metadata_with_ratings['rating'].describe())
    print('number of missing ratings:', metadata_with_ratings['rating'].isnull().sum())
    return metadata_with_ratings

################################## Data exploration. ##################################

def describe_feat(data, feat, as_int=False):
    description = data[feat].describe()
    if as_int:
        description = description.astype(int)
    display(description)
    print(f'number of missing {feat}s:', data[feat].isnull().sum())

def plot_joint_reg(data, x, y, title='', plot_figure=True):
    if plot_figure:
        sns.jointplot(data=data, x=x, y=y, kind='reg',
                      scatter_kws={'alpha': 0.1, 's': 15}, line_kws={'color': 'r'}) \
                .fig.suptitle(title)
    data_cleaned = data.query(f'not {x}.isnull() and not {y}.isnull()')
    X = data_cleaned[x].to_numpy()[:,None]
    model = LinearRegression().fit(X, data_cleaned[y])
    print(f'y = {model.intercept_} + {model.coef_[0]} * x, r^2 = {model.score(X, data_cleaned[y])}')

def plot_pca_variance_ratios(X, n_components=10):
    pca = PCA(n_components=n_components)
    pca.fit(X)
    ratios = pca.explained_variance_ratio_
    x = np.arange(len(ratios))
    plt.plot(x, ratios)
    plt.xlabel('component index')
    plt.ylabel('explained variance ratio')
    plt.title(f'First {n_components} PCA components')

###################################### Modeling. ######################################

def clean_features(metadata_with_ratings, training_data=None, output=True):
    if training_data is None:
        training_data = metadata_with_ratings
    # Clean price.
    metadata_with_ratings['price_float'] = pd.to_numeric(
            metadata_with_ratings['price'].str.replace('$', ''), errors='coerce')
    if output:
        describe_feat(metadata_with_ratings, 'price_float')
    
    # Clean sales rank.
    metadata_with_ratings['rank_float'] = pd.to_numeric(metadata_with_ratings['rank'].str.replace(',', '') \
                                                      .str.extract('^(\d+)', expand=False), errors='coerce')
    if output:
        describe_feat(metadata_with_ratings, 'rank_float', as_int=True)
    
    # Add sales rank category.
    metadata_with_ratings['rank_category'] = metadata_with_ratings['rank'] \
                .str.extract(' in (.+) \(', expand=False) \
                .str.replace('&amp;', '&')
    if output:
        print('categories:')
        print(metadata_with_ratings['rank_category'].value_counts())
    
    # Clean description.
    metadata_with_ratings['description_str'] = metadata_with_ratings['description'].str.join('\n')
    
    # Clean brand.
    brand_counts = training_data['brand'].value_counts().iloc[1:]
    metadata_with_ratings['brand_count'] = pd.to_numeric(metadata_with_ratings['brand'].replace(brand_counts),
                                                         errors='coerce').fillna(0)
    if output:
        describe_feat(metadata_with_ratings, 'brand_count')
    metadata_with_ratings['top_brand'] = (metadata_with_ratings['brand_count'] > 20).astype(int)
    if output:
        print('percentage top brand:', metadata_with_ratings['top_brand'].mean())
    
def transform_col(data, func, feat, new_feat):
    data[new_feat] = func(metadata_with_ratings[feat])
    data[new_feat].describe()

def extract_words(data, feat, max_words=100, encoder=None, output=True):
    if not encoder:
        encoder = CountVectorizer(max_features=max_words, stop_words='english')
        encoder.fit(data[feat])
    X = encoder.transform(data[feat]).toarray()

    if output:
        print('first 50 features:', encoder.get_feature_names()[:50])
        print('feature matrix shape', X.shape)
    return X, encoder

def onehot_encode(data, feat, max_categories=100, encoder=None):
    cat_counts = data[feat].value_counts()
    categories = cat_counts[1:max_categories + 1].index.tolist()
    raw_features = data[feat].to_numpy()[:,None]
    if not encoder:
        encoder = OneHotEncoder(categories=[categories], handle_unknown='ignore', sparse=False)
        encoder.fit(raw_features)
    return encoder.transform(raw_features), encoder

def fill_missing(data, feat, imputer=None):
    if not imputer:
        imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

def standardize(X, scale_columns, encoder=None, output=True):
    X_to_scale = X[:,scale_columns]
    if not encoder:
        encoder = StandardScaler()
        encoder.fit(X_to_scale)
    X[:,scale_columns] = encoder.transform(X_to_scale)
    if output:
        print('mean:', encoder.mean_)
        print('standard deviation:', encoder.scale_)
    return X, encoder

def get_feat_matrix(data, encoders=None):
    if not encoders:
        encoders = [None, None, None, None]
    X1 = data[['price_float', 'brand_count', 'top_brand']].to_numpy()
    X2, encoders[0] = onehot_encode(data, 'brand', encoder=encoders[0])
    X3, encoders[1] = extract_words(data, 'title', encoder=encoders[1], output=False)
    X4, encoders[2] = extract_words(data, 'description_str', 500, encoder=encoders[2], output=False)
    X = np.hstack([X1, X2, X3, X4])
    X, encoders[3] = standardize(X, [0, 1], encoder=encoders[3], output=False)
    return X, encoders

def train_test_split(data, label):
    data_train, data_test, y_train, y_test = \
            model_selection.train_test_split(data, data[label], test_size=0.2, random_state=314)
    clean_features(data_train, training_data=data_train, output=False)
    clean_features(data_test, training_data=data_train, output=False)
    X_train, encoders = get_feat_matrix(data_train)
    X_test, _ = get_feat_matrix(data_test, encoders=encoders)
    print(f'Training data of shape {X_train.shape}, test data {X_test.shape}')
    return X_train, X_test, y_train, y_test

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def mean_proportional_error(y, y_pred):
    return np.sqrt(np.mean(((y - y_pred) / y)**2))

def train_ridge(X_train, X_test, y_train, y_test):
    alpha_exponents = np.arange(-5, 5, 0.2) * np.log(10)
    model = RidgeCV(alphas=np.exp(alpha_exponents))
    return train_model(X_train, X_test, y_train, y_test, model)
    
def train_model(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    train_r2 = model.score(X_train, y_train)
    test_r2 = model.score(X_test, y_test)
    train_loss = rmse(y_train, y_train_pred)
    test_loss = rmse(y_test, y_test_pred)
    train_err = mean_proportional_error(y_train, y_train_pred)
    test_err = mean_proportional_error(y_test, y_test_pred)
    
    print('training r^2:', train_r2)
    print('test r^2:', test_r2)
    print('training loss:', train_loss)
    print('test loss:', test_loss)
    print('training proportional loss:', train_err)
    print('test proportional loss:', test_err)
    return model

## Load data

### Column labels:

Metadata:

- asin - ID of the product, e.g. 0000031852
- title - name of the product
- feature - bullet-point format features of the product
- description - description of the product
- price - price in US dollars (at time of crawl)
- image - url of the product image
- related - related products (also bought, also viewed, bought together, buy after viewing)
- salesRank - sales rank information
- brand - brand name
- categories - list of categories the product belongs to
- tech1 - the first technical detail table of the product
- tech2 - the second technical detail table of the product
- similar - similar product table

Reviews:

- reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
- asin - ID of the product, e.g. 0000013714
- reviewerName - name of the reviewer
- vote - helpful votes of the review
- style - a disctionary of the product metadata, e.g., "Format" is "Hardcover"
- reviewText - text of the review
- overall - rating of the product
- summary - summary of the review
- unixReviewTime - time of the review (unix time)
- reviewTime - time of the review (raw)
- image - images that users post after they have received the product

### Import data

In [ ]:
url = "http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/All_Beauty.json.gz"
filename = 'data/All_Beauty.json.gz'
reviews = load_data(url, filename)

In [ ]:
url = "http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_All_Beauty.json.gz"
filename = 'data/Meta_All_Beauty.json.gz'
metadata = load_data(url, filename)

### Merge reviews and metadata

In [ ]:
metadata_with_ratings = get_metadata_with_ratings(reviews, metadata)

### Clean features

In [ ]:
clean_features(metadata_with_ratings)

In [ ]:
transform_col(metadata_with_ratings, np.log, 'rank_float', 'log_rank')
transform_col(metadata_with_ratings, np.sqrt, 'rank_float', 'sqrt_rank')

### Filter data

In [ ]:
# Remove products with incorrect rank
metadata_beauty = metadata_with_ratings.query('rank_category == "Beauty & Personal Care"')
metadata_beauty.shape

In [ ]:
# Remove products with out price, which likely means the product is no longer available.
metadata_available = metadata_beauty.query('not price_float.isnull()')
metadata_available.shape

## Data exploration

### Sales rank vs average rating

In [ ]:
# The two products with no reviews.
metadata_beauty.query('rating.isnull()')

The following three plots show the correlation between sales rank and average rating. The correlation is slightly different for transformed versions of the rank.

In [ ]:
plot_joint_reg(data=metadata_available, x='log_rank', y='rating', title='Log sales rank vs rating')

In [ ]:
plot_joint_reg(data=metadata_available, x='sqrt_rank', y='rating', title='Square root sales rank vs rating')

In [ ]:
plot_joint_reg(data=metadata_available, x='rank_float', y='rating', title='Sales rank vs rating')

### Price vs sales rank and rating

In [ ]:
price_truncated = metadata_available.query('price_float < 100')
plot_joint_reg(data=price_truncated, x='price_float', y='sqrt_rank')

In [ ]:
plot_joint_reg(data=price_truncated, x='price_float', y='rating')

### Brand

In [ ]:
brand_counts = metadata_available['brand'].value_counts().iloc[1:] # Remove blank ''.
display(brand_counts[:10])
display(brand_counts.describe())

brand_counts_filtered = brand_counts_filtered[brand_counts_filtered > 10]
fig = plt.figure(figsize=(12, 4))
plt.plot(np.arange(len(brand_counts_filtered)), brand_counts_filtered)
plt.ylabel('frequency')
plt.title('Frequency of occurence for brands, from most to least common')

In [ ]:
plot_joint_reg(data=metadata_available, x='brand_count', y='sqrt_rank')

In [ ]:
plot_joint_reg(data=metadata_available, x='brand_count', y='rating')

In [ ]:
sns.boxplot(data=metadata_available, x='top_brand', y='sqrt_rank');

In [ ]:
sns.boxplot(data=metadata_available, x='top_brand', y='rating');

### Title and description

In [ ]:
extract_words(metadata_available, 'title');

In [ ]:
extract_words(metadata_available, 'description_str', 500);

### PCA

In [ ]:
X, encoders = get_feat_matrix(metadata_available)
X.shape

In [ ]:
plot_pca_variance_ratios(X)

In [ ]:
pca = PCA(n_components=2)
components = pca.fit_transform(X)
components_df = pd.DataFrame(data=components, columns=['component1', 'component2'])
data_with_pca = pd.concat([metadata_available.reset_index(drop=True),
                           components_df.reset_index(drop=True)], axis=1)
data_with_pca.shape

In [ ]:
filtered_data = data_with_pca.query('component1 < -0.08 and component2 < -0.2')
print('after filtering:', filtered_data.shape)
sns.jointplot(data=filtered_data, x='component1', y='component2', kind='scatter', alpha=0.1) \
    .fig.suptitle('Distribution of first 2 PCA components (cropped)');

In [ ]:
sns.jointplot(data=data_with_pca.sample(1000), x='component1', y='component2', kind='scatter', alpha=0.1) \
    .fig.suptitle('Distribution of first 2 PCA components (sampled)');

In [ ]:
plot_joint_reg(data=filtered_data, x='component1', y='rank_float',
               title='First principal component vs sales rank')

In [ ]:
plot_joint_reg(data=filtered_data, x='component1', y='rating', title='First principal component vs rating')

In [ ]:
plot_joint_reg(data=filtered_data, x='component2', y='rank_float',
               title='Second principal component vs sales rank')

In [ ]:
plot_joint_reg(data=filtered_data, x='component2', y='rating', title='Second principal component vs rating')

In [ ]:
print('component 1 vs rank')
plot_joint_reg(data=data_with_pca, x='component1', y='rank_float', plot_figure=False)
print('component 1 vs rating')
plot_joint_reg(data=data_with_pca, x='component1', y='rating', plot_figure=False)
print('component 2 vs rank')
plot_joint_reg(data=data_with_pca, x='component2', y='rank_float', plot_figure=False)
print('component 2 vs rating')
plot_joint_reg(data=data_with_pca, x='component2', y='rating', plot_figure=False)

## Modeling

In [ ]:
label = 'sqrt_rank'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(metadata_available, label)

### Ridge regression

In [ ]:
model = train_ridge(X_train, X_test, y_train, y_test)

In [ ]:
np.round(model.coef_ * 10).astype(int)

### LASSO

In [ ]:
model = train_model(X_train, X_test, y_train, y_test, LassoCV(tol=0.1))

### Support vector machine

In [ ]:
# May take several minutes to train.
model = train_model(X_train, X_test, y_train, y_test, SVR(C=250))

### Random forest

In [ ]:
# Setting n_estimators to be greater increases r^2 slightly but takes much longer, so I just went with
# the default. Set n_jobs to make it faster.
model = train_model(X_train, X_test, y_train, y_test, RandomForestRegressor(max_depth=25, n_jobs=4))

### Gradient boosting

In [ ]:
model = train_model(X_train, X_test, y_train, y_test,
                    GradientBoostingRegressor(n_estimators=200, max_depth=5, min_samples_leaf=3))

### Adaboost

In [ ]:
model = train_model(X_train, X_test, y_train, y_test, AdaBoostRegressor())

### Evaluation (common to all models)

In [ ]:
describe_feat(metadata_available, label, as_int=True)

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
for i in range(10):
    print(f'{y_train.iloc[i]:7.0f}, {y_train_pred[i]:7.0f}')
print()
for i in range(10):
    print(f'{y_test.iloc[i]:7.0f}, {y_test_pred[i]:7.0f}')

In [ ]:
residuals = (y_train_pred - y_train) / np.abs(y_train)
sns.histplot(residuals[residuals < 6])

In [ ]:
residuals = (y_test_pred - y_test) / np.abs(y_test)
sns.histplot(residuals[residuals < 6])

In [ ]:
encoders[0].get_feature_names()